## Cloning repository

In [ ]:
%cd /content/
!rm -r ./Speech_Commands_with_RNNs/
!git clone https://github.com/MaroonBlue/Speech_Commands_with_RNNs.git
%cd Speech_Commands_with_RNNs/

## Installing packages

In [ ]:
!pip install -r requirements.txt
!pip install -U -q tensorflow tensorflow_datasets

# Downloading dataset

## Download form URL

In [ ]:
!python utils/download_dataset.py

In [ ]:
!python utils/download_test_dataset.py

In [ ]:
!python ./utils/dataset.py

In [ ]:
!cp ./saved_data/X_t.npy ../X_t.npy
!cp ./saved_data/y_t.npy ../y_t.npy
!cp ./saved_data/X_v.npy ../X_v.npy
!cp ./saved_data/y_v.npy ../y_v.npy
!cp ./saved_data/X_test.npy ../X_test.npy
!cp ./saved_data/y_test.npy ../y_test.npy

In [ ]:
!cp ../X_t.npy ./saved_data/X_t.npy
!cp ../y_t.npy ./saved_data/y_t.npy
!cp ../X_v.npy ./saved_data/X_v.npy
!cp ../y_v.npy ./saved_data/y_v.npy
!cp ../X_test.npy ./saved_data/X_test.npy
!cp ../y_test.npy ./saved_data/y_test.npy

In [ ]:
import os
import wandb
import tensorflow as tf
import sys
import numpy as np

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from numpy import load

sys.path.append("./")
from utils.utils import set_seeds, make_configs, step_decay
from models.test_model import get_test_model
from models.CNN_LSTM import CNN_LSTM1
from models.GRU import GRU1, GRU2, GRU3
from models.LSTM import LSTM1, LSTM2, LSTM3
from models.simpleRNN import simpleRNN1, simpleRNN2, simpleRNN3

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
ENTITY = 'wo-team'
PROJECT = 'dl-rnn-audio'
GROUP = 'simple-lstm-gru'
NAME = '_'
SAVE_PATH = 'weights/'

models = {
    'GRU3': GRU3,
}
base_config = {
    'dataloader': {
        'sample_rate': 8000,
        'unknown_silence_samples': 2000,
        'seed': 0,
        'batch_size': 128,
        'convert_to_image': False,
    },
    'training': {
        'n_epochs': 50,
        'dropout': 0.3,
    },
    'compile':{
        'loss': 'sparse_categorical_crossentropy',
        'optimizer': 'adam',
        'metrics': ['accuracy', 'sparse_categorical_accuracy']
    },
    'model': {
        'architecture': 'GRU3',
        'model_init': None,
        'id': None,
        'save_path': None,
    },
    'early_stopper':{
        'monitor': 'val_sparse_categorical_accuracy',
        'min_delta': 0.001,
        'patience': 3,
        'verbose': 1,
        'start_from_epoch': 5,
        'restore_best_weights': True,
    },
    'checkpointer':{
        'monitor': 'val_sparse_categorical_accuracy',
        "verbose": 1,
        'save_best_only': True
    },
    'scheduler': LearningRateScheduler(step_decay),
    'other':{
            'num_classes':12,
    }
}

combinations = {
    'seeds': {
        'dict_path': ['dataloader', 'seed'],
        'values': [0]
    }

}



configs = make_configs(base_config, combinations)

LOAD_PTH = "./saved_data/"
X_t = load(LOAD_PTH + "X_t.npy")
y_t = load(LOAD_PTH + "y_t.npy")
X_v = load(LOAD_PTH + "X_v.npy")
y_v = load(LOAD_PTH + "y_v.npy")
y_t = np.argmax(y_t, axis=1).transpose()
y_v = np.argmax(y_v, axis=1).transpose()

In [ ]:
best_model = tf.keras.models.load_model('0GRU3_seed0_dropout0,3.h5')

In [ ]:
config = base_config
wandb.init(
    project = PROJECT,
    entity = ENTITY,
    group = GROUP,
    name = NAME,
    config = config)

input_shape = X_t.shape[1:]

earlystopper = EarlyStopping(**config["early_stopper"])
checkpointer = ModelCheckpoint(NAME+'.h5', **config["checkpointer"])
lrate = config["scheduler"]

history = best_model.fit(
            X_t,
            y_t,
            epochs=config['training']['n_epochs'],
            validation_data=(X_v, y_v),
            batch_size=config['dataloader']['batch_size'],
            shuffle=True,
            callbacks=[
                earlystopper, 
                checkpointer, 
                lrate,
                WandbMetricsLogger(log_freq=5),
                WandbModelCheckpoint("weights/wandb")
        ])
save_path = os.path.join(SAVE_PATH, NAME)
best_model.save(save_path)

wandb.finish()